In [1]:
ip='172.31.19.202'
db='amazon_sp_api'
password='Techblooprint123'
user="blooprint"


In [2]:
from pyspark.sql import SparkSession
import os
from clickhouse_driver import Client

spark = SparkSession.builder \
    .appName("BigCSVWrite") \
    .config("spark.driver.memory", "5g") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.shuffle.partitions", "50")
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame  # or: from pyspark.sql.dataframe import DataFrame
from pyspark.sql import functions as F, types as T
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import broadcast
from pyspark.sql import functions as F
# Should not raise:
spark.sparkContext._jvm.java.lang.Class.forName("org.postgresql.Driver")

URL  = os.getenv("PG_JDBC_URL", f"jdbc:postgresql://{ip}:5433/{db}")
USER = os.getenv("PG_USER", user)
PWD  = os.getenv("PG_PASSWORD", password)
portfolio_query = "(SELECT * FROM public.portfolio_id_name_mapping) AS p"

portfolio_id_name_mapping = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", portfolio_query)
      .load())



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/17 07:52:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/17 07:52:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
sp_advertised_query = """
(SELECT account_name,
            "advertisedAsin" AS asin,
            "campaignId" AS campaign_id,
            "campaignName" AS campaign_name,
            "adGroupName" AS ad_group_name,
            date AS date,
            impressions,
            clicks,
            spend AS ads_spend,
            sales30d AS ads_sales,
            "unitsSoldClicks30d" AS units
           FROM public.sp_advertised_product) as ad_p

"""

sp_advertised_df = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", sp_advertised_query)
      .load())
sp_advertised_df=sp_advertised_df.withColumn("date",
    F.col("date").cast(T.DateType())
)

In [4]:
sd_advertised_query = """
(SELECT account_name,
            "promotedAsin" AS asin,
            "campaignId" AS campaign_id,
            "campaignName" AS campaign_name,
            "adGroupName" AS ad_group_name,
            date AS date,
            impressions,
            clicks,
            cost AS ads_spend,
            sales AS ads_sales,
            "unitsSold" AS units
           FROM public.sd_advertised_product) as sd_advertised_query
"""

sd_advertised_df = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", sd_advertised_query)
      .load())
sd_advertised_df=sd_advertised_df.withColumn("date",
    F.col("date").cast(T.DateType())
)

In [5]:
sb_advertised_query = """
(select account_name,
            "purchasedAsin" AS asin,
            "campaignId" AS campaign_id,
            "campaignName" AS campaign_name,
            "adGroupName" AS ad_group_name,
            date AS date,
            NULL::numeric AS impressions,
            NULL::numeric AS clicks,
            NULL::numeric AS ads_spend,
            sales14d AS ads_sales,
            "unitsSold14d" AS units from public.sb_purchased_product) as sb_advertised_query
"""

sb_advertised_df = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", sb_advertised_query)
      .load())
sb_advertised_df=sd_advertised_df.withColumn("date",
    F.col("date").cast(T.DateType())
)

In [6]:
advertised_asin=sp_advertised_df.unionByName(sb_advertised_df).unionByName(sd_advertised_df)

In [7]:
advertised_asin=advertised_asin.withColumn("campaign_id",
                           F.split(
                               F.col("campaign_id").cast(T.StringType()),r'\.'
                               )[0].astype("long")
                            )

In [8]:
campaign_query = "(SELECT * FROM public.campaigns) as c"
campaigns = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", campaign_query)
      .load())

campaign_df=campaigns.select(F.col("campaignId").alias("campaign_id"),
                 F.col("portfolioId").alias("portfolioId")
                 ).dropDuplicates(['campaign_id'])
portfolio_df=portfolio_id_name_mapping.select(F.col("portfolioId").alias("portfolioId"),
                                 F.col("name").alias("portfolio_name")).dropDuplicates(['portfolioId'])
portfolio_mapping=campaign_df.join(portfolio_df,on='portfolioId',how='left')

In [9]:
portfolio_mapping=portfolio_mapping.drop_duplicates()

In [10]:
asin_level_data=portfolio_mapping.join(advertised_asin,how='outer',on="campaign_id")

In [11]:
asin_data=asin_level_data.fillna({"impressions": 0,
                "clicks":0,
                 "ads_spend":0,
                 "UNITS":0,
                 "ads_sales":0,
                 "units":0,
                }).select(
                    F.col("account_name").alias("ACCOUNT NAME"),
                    F.col("asin").alias("ASINS"),
                    F.col("portfolio_name").alias("PORTFOLIOS"),
                    F.col("campaign_name").alias("CAMPAIGNS"),
                    F.col("ad_group_name").alias("AD GRP"),

                    F.col("date").alias("DATE"),
                    F.col("impressions").astype("int").alias("IMPRSN"),
                    F.col("clicks").astype("int").alias("CLICKS"),
                    F.col("ads_spend").astype("int").alias("SPEND"),
                    F.col("ads_sales").astype("int").alias("SALES"),
                    F.col("units").astype("int").alias("UNITS")

                )

In [12]:
dummy_asin_data=asin_data.filter(F.col("ACCOUNT NAME")=='awenest')

In [13]:
pattern = r"(?i)\bawenest\b"   
string_cols = [c for c, t in dummy_asin_data.dtypes if t == "string"]

dummy_asin_data = dummy_asin_data.select([
    F.regexp_replace(F.col(c), pattern, "Dummy").alias(c) if c in string_cols else F.col(c)
    for c, _ in dummy_asin_data.dtypes
])

In [ ]:
# dummy_asin_data.show()

+------------+----------+--------------------+--------------------+--------------------+----------+------+------+-----+-----+-----+
|ACCOUNT NAME|     ASINS|          PORTFOLIOS|           CAMPAIGNS|              AD GRP|      DATE|IMPRSN|CLICKS|SPEND|SALES|UNITS|
+------------+----------+--------------------+--------------------+--------------------+----------+------+------+-----+-----+-----+
|       Dummy|B0CF9Q33MY|Dishwash powder t...|Dishwash Powder-B...|Dishwash Powder-B...|2024-09-23|     0|     0|    0|    0|    0|
|       Dummy|B0CRWHYN78|Dishwash powder t...|Dishwash Powder-B...|Dishwash Powder-B...|2024-09-23|     0|     0|    0|    0|    0|
|       Dummy|B0CF9Q33MY|Dishwash powder t...|Dishwash Powder-B...|Dishwash Powder-B...|2024-09-23|     0|     0|    0|    0|    0|
|       Dummy|B0CRWHYN78|Dishwash powder t...|Dishwash Powder-B...|Dishwash Powder-B...|2024-09-23|     0|     0|    0|    0|    0|
|       Dummy|B0BXDR5644|   Bamboo Toothbrush|Toothbrush-BOBXDR...| Toothbru

In [14]:
from pyspark.sql.window import Window

# Step 1: Rename AD GRP → AD_GRP for processing
df = dummy_asin_data.withColumnRenamed("AD GRP", "AD_GRP")

w_asin = Window.orderBy("ASINS")
asin_map = (
    df.select("ASINS").distinct()
      .withColumn("ASINS_NEW", F.concat(F.lit("asin"), F.dense_rank().over(w_asin)))
)
df = df.join(asin_map, on="ASINS", how="left").drop("ASINS").withColumnRenamed("ASINS_NEW", "ASINS")

# --- PORTFOLIOS ---
w_portfolio = Window.orderBy("PORTFOLIOS")
portfolio_map = (
    df.select("PORTFOLIOS").distinct()
      .withColumn("PORTFOLIOS_NEW", F.concat(F.lit("portfolio"), F.dense_rank().over(w_portfolio)))
)
df = df.join(portfolio_map, on="PORTFOLIOS", how="left").drop("PORTFOLIOS").withColumnRenamed("PORTFOLIOS_NEW", "PORTFOLIOS")

# --- CAMPAIGNS ---
w_campaign = Window.partitionBy("PORTFOLIOS").orderBy("CAMPAIGNS")
campaign_map = (
    df.select("PORTFOLIOS", "CAMPAIGNS").distinct()
      .withColumn("CAMPAIGNS_NEW", F.concat(F.lit("campaign"), F.dense_rank().over(w_campaign)))
)
df = df.join(campaign_map, on=["PORTFOLIOS", "CAMPAIGNS"], how="left").drop("CAMPAIGNS").withColumnRenamed("CAMPAIGNS_NEW", "CAMPAIGNS")

# --- AD GROUPS ---
w_adgroup = Window.partitionBy("PORTFOLIOS", "CAMPAIGNS").orderBy("AD_GRP")
adgroup_map = (
    df.select("PORTFOLIOS", "CAMPAIGNS", "AD_GRP").distinct()
      .withColumn("AD_GRP_NEW", F.concat(F.lit("adgroup"), F.dense_rank().over(w_adgroup)))
)
df = df.join(adgroup_map, on=["PORTFOLIOS", "CAMPAIGNS", "AD_GRP"], how="left").drop("AD_GRP").withColumnRenamed("AD_GRP_NEW", "AD_GRP")

# Step 2: Multiply numeric metrics
cols_to_multiply = ["IMPRSN", "CLICKS", "SPEND", "SALES", "UNITS"]
for c in cols_to_multiply:
    df = df.withColumn(c, F.col(c).cast("double") * 5)



# Step 4: Rename AD_GRP back
df = df.withColumnRenamed("AD_GRP", "AD GRP")


In [16]:
df.show()

25/09/17 07:49:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/17 07:49:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/17 07:49:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/17 07:49:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/17 07:49:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/17 07:49:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/17 0

+-----------+----------+------------+----------+------+------+-----+-----+-----+------+--------+----+---+----+---+
| PORTFOLIOS| CAMPAIGNS|ACCOUNT NAME|      DATE|IMPRSN|CLICKS|SPEND|SALES|UNITS| ASINS|  AD GRP| CPC|ROI|ACOS|CVR|
+-----------+----------+------------+----------+------+------+-----+-----+-----+------+--------+----+---+----+---+
| portfolio8|campaign42|       Dummy|2025-01-02|  10.0|   0.0|  0.0|  0.0|  0.0|asin55|adgroup2| 0.0|0.0| 0.0|0.0|
| portfolio8|campaign42|       Dummy|2025-01-03|   5.0|   0.0|  0.0|  0.0|  0.0|asin55|adgroup2| 0.0|0.0| 0.0|0.0|
| portfolio8|campaign42|       Dummy|2025-01-06|   5.0|   0.0|  0.0|  0.0|  0.0|asin55|adgroup2| 0.0|0.0| 0.0|0.0|
| portfolio8|campaign42|       Dummy|2025-01-07|   5.0|   0.0|  0.0|  0.0|  0.0|asin55|adgroup2| 0.0|0.0| 0.0|0.0|
| portfolio8|campaign42|       Dummy|2024-12-28|   5.0|   0.0|  0.0|  0.0|  0.0|asin55|adgroup2| 0.0|0.0| 0.0|0.0|
| portfolio8|campaign42|       Dummy|2025-01-04|  25.0|   0.0|  0.0|  0.0|  0.0|

In [15]:
final_asin_data=asin_data.unionByName(df)

In [16]:
(
    final_asin_data.write
    .format("jdbc")
    .option("url", URL)
    .option("user", USER)
    .option("password", PWD)
    .option("driver", "org.postgresql.Driver")
    .option("dbtable", "public.asin_info")  # destination table
    .mode("overwrite")  # creates table if not exists, replaces if exists
    .save()
)


25/09/17 07:52:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/09/17 07:52:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/17 07:52:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/17 07:52:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/17 07:52:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/17 07:52:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance